### Week 3 assignment - Neighbourhoods in Toronto

We will analyze and cluster neighbourhoods in Toronto

In [1]:
#Importing all the needed packages

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium

<p> Uncomment and install as required:

In [2]:
#!conda install -c conda-forge lxml --yes #Install LXML package to use with Pandas HTML parser.
#!conda install -c conda-forge folium=0.5.0 --yes #Install Folium to draw maps

### Getting the Toronto data from the Wikipedia page

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
all_tables = pd.read_html(str(table))
#print( tabulate(all_tables[0], headers='keys', tablefmt='psql') )

In [4]:
# all_tables contains all the tables found in the page, the first table at index 0 contains our target table
df = all_tables[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Pre-processing

Revome rows were the borough is not assigned:

In [5]:
df = df[df.Borough != 'Not assigned']

Assign the Borough value to the Neighbourhood when the Neighbourhood is not assigned:

In [6]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df['Borough']

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Group neighbourhoods of the same Borough:

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Size of resulting data frame: 

In [8]:
print('The resulting dataframe has', df.shape[0] , ' rows and ', df.shape[1] , ' columns.')

The resulting dataframe has 103  rows and  3  columns.


### Let's get the coordinates data

In [9]:
coord_data = pd.read_csv('http://cocl.us/Geospatial_data')

In [10]:
coord_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<p>Adding the coordinates values to the dataframe by merging the two dataframes, the neighbourhoods dataframe and the coordinates dataframe: </p>

In [11]:
df = df.merge(coord_data, how='inner',left_on = 'Postcode', right_on = 'Postal Code' )

In [12]:
#Remmove column Postal Code as it exists twice
df.drop('Postal Code', axis=1, inplace=True)

In [13]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<p> We will only work with boroughs that have the word Toronto in their names:

In [14]:
toronto_neighborhoods = df[df['Borough'].str.contains("Toronto")]

#toronto_neighborhoods = df

In [15]:
toronto_neighborhoods.shape

(38, 5)

<p> Let's get the Toronto longitude and latitude

In [16]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


# Analysis and visualization

Let's visualize the neighbourhoods that we will be working with:

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
# @hidden_cell

CLIENT_ID = 'T3X5JYPNZARTTO5FHG5WJ5024XALTH4X1NDEVUCPRF5CLTB2' # your Foursquare ID
CLIENT_SECRET = 'AUNSRZCFLE5VXJ0O5JNUCMTMYI4FXXYKEVKJUUZ3A2CGQRNO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

<p> Let's retrieve venues lists from Foursquare:

In [19]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighbourhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1715, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Find the number of venues in each neighbourhood

In [22]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",27,27,27,27,27,27
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Let's find how many unique categories there are: 

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [25]:
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_onehot.shape

(1715, 241)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.01,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.00,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.074074
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3              Bar  0.04
4            Hotel  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2  Farmers Market  0.04
3      Steakhouse  0.04
4            Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0            Café  0.11
1     Yoga Studio  0.07
2          Bakery  0.07
3     Coffee Shop  0.07
4  Breakfast Spot  0.07


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Auto Workshop  0.05
3                Park  0.05
4          Comic Shop  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3  

This function sorts the venues in descending order:

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
import numpy as np 
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Hotel,Sushi Restaurant,Asian Restaurant,Steakhouse,American Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Seafood Restaurant,Beer Bar,Farmers Market,Steakhouse,Bakery,Cheese Shop,Fish Market
2,"Brockton, Exhibition Place, Parkdale Village",Café,Yoga Studio,Coffee Shop,Breakfast Spot,Bakery,Burrito Place,Sandwich Place,Caribbean Restaurant,Stadium,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Pizza Place,Restaurant,Butcher,Burrito Place,Skate Park,Brewery,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Bar,Airport Gate


### Clustering

#### We will run k-means to create 7 clusters:

In [31]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 6, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
toronto_merged = toronto_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')



In [34]:
toronto_merged.head(20) 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Other Great Outdoors,Pub,Neighborhood,Trail,Event Space,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Juice Bar,Sports Bar,Spa,Liquor Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Pizza Place,Pet Store,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Park,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Fish Market,Bookstore,Latin American Restaurant,Brewery,Seafood Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Pizza Place,Convenience Store,Breakfast Spot,Gym,Park,Hotel,Sandwich Place,Food & Drink Shop,Dance Studio,Clothing Store
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Park,Spa,Metro Station,Burger Joint,Mexican Restaurant,Salon / Barbershop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Coffee Shop,Pizza Place,Gym,Italian Restaurant,Farmers Market,Diner
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Playground,Tennis Court,Restaurant,Gym,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1,Coffee Shop,Pub,Pizza Place,Bagel Shop,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Supermarket


##### Coffee shops seem to be the most common venue is several neighborhoods. 
Let's take a look at the first 3 clusters:

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Building,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Trail,Jewelry Store,Park,Bus Line,Sushi Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Juice Bar,Sports Bar,Spa,Liquor Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Pizza Place,Pet Store,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Park,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Fish Market,Bookstore,Latin American Restaurant,Brewery,Seafood Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Pizza Place,Convenience Store,Breakfast Spot,Gym,Park,Hotel,Sandwich Place,Food & Drink Shop,Dance Studio,Clothing Store
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Park,Spa,Metro Station,Burger Joint,Mexican Restaurant,Salon / Barbershop
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Sushi Restaurant,Café,Coffee Shop,Pizza Place,Gym,Italian Restaurant,Farmers Market,Diner
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,1,Coffee Shop,Pub,Pizza Place,Bagel Shop,Fried Chicken Joint,Light Rail Station,Liquor Store,Restaurant,Sports Bar,Supermarket
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Bakery,Coffee Shop,Café,Italian Restaurant,Restaurant,Pub,Pizza Place,Pet Store,Pharmacy,Butcher
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Café,Pub,Bakery,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Spa


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Other Great Outdoors,Pub,Neighborhood,Trail,Event Space,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant


Now let's visualize the result:

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters